In [10]:
# Import this library
from nbacktest import Backtest, Strategy, RealBroker

# Import other libraries
import yfinance as yf
import warnings

# Ignore warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [11]:
universe = ["COGN3.SA", "BBDC3.SA"] # List containing the tickers of all the assets you are backtesting

df_ohlc = yf.download(universe, start="2022-01-01", end="2023-01-01", interval="1d") # Download df_ohlc data from Yahoo Finance

print(df_ohlc.shape)
df_ohlc.tail()

# Rename ticker level (level=1)
df_ohlc = df_ohlc.rename(
    columns=lambda x: x.replace(".SA", ""),
    level=1
)

universe = ["COGN3", "BBDC3"] 

df_ohlc = df_ohlc.tail()

[*********************100%***********************]  2 of 2 completed

(250, 10)


In [12]:
class TestStrategy (Strategy):

    def on_start(self):

        print("Strategy started!")
        
        quantity1 = 100

        order1 = self.buy("BBDC3", quantity1)

        trade1 = self.create_trade(orders=[order1], notes="Real Trade Example")

        print("Bought %s" % (quantity1))


    def on_end(self):
        """"
        This function is called when the backtest of all candles is finished
        """
        print("Strategy finished!")


    def next(self):
        """"
        This function is called every candle (iteration)
        """
        #print("Backtest iteration:", self.iteration)
        pass

In [13]:
from nbacktest.core.broker import RealBroker

base_url = "http://localhost:8081/"
real_broker = RealBroker(universe=universe, cash=10_000, base_url=base_url)

bt_real = Backtest(data=df_ohlc,
                   universe=universe,
                   strategy_class=TestStrategy,
                   price_column="Close",
                   cash=10_000,
                   broker=real_broker)

result_real = bt_real.run()
result_real.head()

Strategy started!
Order placement failed: HTTP 400 {"detail":{"retcode":10018,"deal":0,"order":0,"volume":0.0,"price":0.0,"bid":0.0,"ask":0.0,"comment":"Market closed","request_id":102329828,"retcode_external":0,"request":{"action":1,"magic":20250,"order":0,"symbol":"BBDC3","volume":100.0,"price":16.3,"stoplimit":0.0,"sl":0.0,"tp":0.0,"deviation":1,"type":0,"type_filling":0,"type_time":0,"expiration":0,"comment":"nbacktest","position":0,"position_by":0}}}


RuntimeError: Order placement failed: HTTP 400 {"detail":{"retcode":10018,"deal":0,"order":0,"volume":0.0,"price":0.0,"bid":0.0,"ask":0.0,"comment":"Market closed","request_id":102329828,"retcode_external":0,"request":{"action":1,"magic":20250,"order":0,"symbol":"BBDC3","volume":100.0,"price":16.3,"stoplimit":0.0,"sl":0.0,"tp":0.0,"deviation":1,"type":0,"type_filling":0,"type_time":0,"expiration":0,"comment":"nbacktest","position":0,"position_by":0}}}

In [5]:
df_ohlc

Price           Close                 High                  Low            \
Ticker          BBDC3     COGN3      BBDC3     COGN3      BBDC3     COGN3   
Date                                                                        
2022-12-23  10.209589  2.112774  10.255441  2.141983  10.049109  2.044620   
2022-12-26   9.972688  2.103038  10.156093  2.132246   9.942120  2.054356   
2022-12-27  10.033823  2.005675  10.186661  2.112774   9.903911  2.005675   
2022-12-28  10.339502  2.103038  10.377712  2.112774  10.026184  2.005675   
2022-12-29  10.293648  2.064092  10.469411  2.141983  10.270722  2.034883   

Price            Open              Volume            
Ticker          BBDC3     COGN3     BBDC3     COGN3  
Date                                                 
2022-12-23  10.117886  2.064092   3177800  21629400  
2022-12-26  10.110242  2.122510   2063500   5967100  
2022-12-27  10.186661  2.112774  26504100  16932500  
2022-12-28  10.087319  2.015411  23236400  15476500  
2022-12-29  10.370066  2.112774  14129200  20212700

In [7]:
bt_real.tradebook

,ID,STATUS,NOTES,PNL,CREATED_ITERATION,CLOSED_ITERATION,REASON_CLOSED,POSITIONS_FILLED,POSITIONS_FILLED_TOTAL,POSITIONS_UNFILLED,POSITIONS_UNFILLED_TOTAL,STOP_LOSS,TAKE_PROFIT,MAX_AGE,AGE
0,9d5d40d6-96e1-43d9-ad51-3dd7344b6df4,OPEN,Real Trade Example,1029.364777,2022-12-23,None,None,"{'BBDC3': {'quantity': 100.0, 'value': 1029.36...",1029.364777,{},0,None,None,None,6 days


In [9]:
bt_real.orderbook

,ID,TICKER,ACTION,STATUS,TRADE_ID,REQUESTED_ITERATION,REQUESTED_QUANTITY,REQUESTED_PRICE,REQUESTED_FEE,REQUESTED_GROSS_TOTAL,REQUESTED_TOTAL,FILLED_ITERATION,FILLED_QUANTITY,FILLED_PRICE,FILLED_FEE,FILLED_GROSS_TOTAL,FILLED_TOTAL,PROVIDER_ID
0,4219525a-27f7-4839-abd5-af644443b9fa,BBDC3,BUY,FILLED,9d5d40d6-96e1-43d9-ad51-3dd7344b6df4,2022-12-23,100,16.3,0.0,1630.0,1630.0,2022-12-26,100.0,16.3,0.0,-1630.0,-1630.0,2298990220
